## Unzip the data

In [1]:
!gunzip ./data/train/* ./data/test/*

gzip: ./data/train/train_identity.csv: unknown suffix -- ignored
gzip: ./data/train/train_transaction.csv: unknown suffix -- ignored
gzip: ./data/test/test_identity.csv: unknown suffix -- ignored
gzip: ./data/test/test_transaction.csv: unknown suffix -- ignored


## Imports

In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.precision = 15

import time
import datetime

import gc
import seaborn as sns

## Constants

In [3]:
train_path = './data/train/'
test_path = './data/test/'

## Raw Training Data

In [4]:
train_identity = pd.read_csv(f'{train_path}train_identity.csv')
train_transaction = pd.read_csv(f'{train_path}train_transaction.csv')

In [5]:
train_identity.describe()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,1.442330000000000e+05,144233.000000000000000,140872.000000000000000,66324.000000000000000,66324.000000000000000,136865.000000000000000,136865.000000000000000,5155.000000000000000,5155.000000000000000,74926.000000000000000,...,139369.000000000000000,45113.000000000000000,139318.000000000000000,139261.000000000000000,5159.000000000000000,5169.000000000000000,4747.000000000000000,5132.000000000000000,5163.000000000000000,77586.000000000000000
mean,3.236329311287985e+06,-10.170501896237338,174716.584708103822777,0.060189373379169,-0.058937941016826,1.615584700252073,-6.698710408066343,13.285354025218234,-38.600387972841901,0.091023142834263,...,189.451377279021870,14.237337352869462,353.128174392397227,403.882666360287544,368.269819732506278,16.002708454246470,12.800926901200759,329.608924395946985,149.070307960488094,26.508596911813989
std,1.788495711856579e+05,14.347949101433478,159651.816855696262792,0.598231339461272,0.701015304106987,5.249856482497284,16.491104461198184,11.384206770817135,26.084899144388590,0.983841572502770,...,30.375360488609033,1.561301528050232,141.095343019131320,152.160327216310918,198.847038472717429,6.897664752694577,2.372446657786935,97.461088799114691,32.101994679765170,3.737502025101397
min,2.987004000000000e+06,-100.000000000000000,1.000000000000000,-13.000000000000000,-28.000000000000000,-72.000000000000000,-100.000000000000000,-46.000000000000000,-100.000000000000000,-36.000000000000000,...,100.000000000000000,10.000000000000000,100.000000000000000,100.000000000000000,100.000000000000000,10.000000000000000,11.000000000000000,100.000000000000000,100.000000000000000,0.000000000000000
25%,3.077142000000000e+06,-10.000000000000000,67992.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,-6.000000000000000,5.000000000000000,-48.000000000000000,0.000000000000000,...,166.000000000000000,13.000000000000000,266.000000000000000,256.000000000000000,252.000000000000000,14.000000000000000,11.000000000000000,321.000000000000000,119.000000000000000,24.000000000000000
50%,3.198818000000000e+06,-5.000000000000000,125800.500000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,14.000000000000000,-34.000000000000000,0.000000000000000,...,166.000000000000000,15.000000000000000,341.000000000000000,472.000000000000000,252.000000000000000,14.000000000000000,11.000000000000000,321.000000000000000,149.000000000000000,24.000000000000000
75%,3.392923000000000e+06,-5.000000000000000,228749.000000000000000,0.000000000000000,0.000000000000000,1.000000000000000,0.000000000000000,22.000000000000000,-23.000000000000000,0.000000000000000,...,225.000000000000000,15.000000000000000,427.000000000000000,533.000000000000000,486.500000000000000,14.000000000000000,15.000000000000000,371.000000000000000,169.000000000000000,32.000000000000000
max,3.577534000000000e+06,0.000000000000000,999595.000000000000000,10.000000000000000,0.000000000000000,52.000000000000000,0.000000000000000,61.000000000000000,0.000000000000000,25.000000000000000,...,229.000000000000000,29.000000000000000,671.000000000000000,661.000000000000000,854.000000000000000,44.000000000000000,26.000000000000000,548.000000000000000,216.000000000000000,32.000000000000000


In [6]:
len(train_identity.TransactionID.unique())

144233

In [7]:
train_transaction.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400000000000e+05,590540.000000000000000,5.905400000000000e+05,590540.000000000000000,590540.000000000000000,581607.000000000000000,588975.000000000000000,586281.000000000000000,524834.000000000000000,524834.000000000000000,...,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000,82351.000000000000000
mean,3.282269500000000e+06,0.034990009144173,7.372311310116165e+06,135.027176372472638,9898.734658109526208,362.555488499966486,153.194925081709755,199.278896979434762,290.733793923411952,86.800630294531217,...,0.775874002744350,721.741882916404506,1375.783644036513351,1014.622781509143010,9.807015093927214,59.164549671486512,28.530902964203371,55.352421950538890,151.160542070513145,100.700881592801892
std,1.704743583211270e+05,0.183754634178414,4.617223646539683e+06,239.162522013734019,4901.170153494897932,157.793246310044651,11.336443990207302,41.244452965022390,101.741071758171117,2.690623486052387,...,4.727971393408287,6217.223583406429498,11169.275702404978801,7955.735481948921006,243.861390570583268,387.629479999626142,274.576919719621969,668.486832619135953,1095.034386884882679,814.946721848684888
min,2.987000000000000e+06,0.000000000000000,8.640000000000000e+04,0.251000000000000,1000.000000000000000,100.000000000000000,100.000000000000000,100.000000000000000,100.000000000000000,10.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000
25%,3.134634750000000e+06,0.000000000000000,3.027057750000000e+06,43.321000000000005,6019.000000000000000,214.000000000000000,150.000000000000000,166.000000000000000,204.000000000000000,87.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000
50%,3.282269500000000e+06,0.000000000000000,7.306527500000000e+06,68.769000000000005,9678.000000000000000,361.000000000000000,150.000000000000000,226.000000000000000,299.000000000000000,87.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000
75%,3.429904250000000e+06,0.000000000000000,1.124662000000000e+07,125.000000000000000,14184.000000000000000,512.000000000000000,150.000000000000000,226.000000000000000,330.000000000000000,87.000000000000000,...,0.000000000000000,0.000000000000000,25.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000
max,3.577539000000000e+06,1.000000000000000,1.581113100000000e+07,31937.390999999999622,18396.000000000000000,600.000000000000000,231.000000000000000,237.000000000000000,540.000000000000000,102.000000000000000,...,55.000000000000000,160000.000000000000000,160000.000000000000000,160000.000000000000000,55125.000000000000000,55125.000000000000000,55125.000000000000000,104060.000000000000000,104060.000000000000000,104060.000000000000000


In [8]:
len(train_transaction.TransactionID.unique())

590540

## Joined Training Data

In [9]:
train = pd.merge(
    train_transaction,
    train_identity,
    on='TransactionID',
    how='left'
)

In [10]:
train.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,5.905400000000000e+05,590540.000000000000000,5.905400000000000e+05,590540.000000000000000,590540.000000000000000,581607.000000000000000,588975.000000000000000,586281.000000000000000,524834.000000000000000,524834.000000000000000,...,139369.000000000000000,45113.000000000000000,139318.000000000000000,139261.000000000000000,5159.000000000000000,5169.000000000000000,4747.000000000000000,5132.000000000000000,5163.000000000000000,77586.000000000000000
mean,3.282269500000000e+06,0.034990009144173,7.372311310116165e+06,135.027176372472638,9898.734658109526208,362.555488499966486,153.194925081709755,199.278896979434762,290.733793923411952,86.800630294531217,...,189.451377279021870,14.237337352869462,353.128174392397227,403.882666360287544,368.269819732506278,16.002708454246470,12.800926901200759,329.608924395946985,149.070307960488094,26.508596911813989
std,1.704743583211270e+05,0.183754634178414,4.617223646539683e+06,239.162522013734019,4901.170153494897932,157.793246310044651,11.336443990207302,41.244452965022390,101.741071758171117,2.690623486052387,...,30.375360488609029,1.561301528050232,141.095343019131292,152.160327216310918,198.847038472717458,6.897664752694577,2.372446657786936,97.461088799114719,32.101994679765170,3.737502025101396
min,2.987000000000000e+06,0.000000000000000,8.640000000000000e+04,0.251000000000000,1000.000000000000000,100.000000000000000,100.000000000000000,100.000000000000000,100.000000000000000,10.000000000000000,...,100.000000000000000,10.000000000000000,100.000000000000000,100.000000000000000,100.000000000000000,10.000000000000000,11.000000000000000,100.000000000000000,100.000000000000000,0.000000000000000
25%,3.134634750000000e+06,0.000000000000000,3.027057750000000e+06,43.321000000000005,6019.000000000000000,214.000000000000000,150.000000000000000,166.000000000000000,204.000000000000000,87.000000000000000,...,166.000000000000000,13.000000000000000,266.000000000000000,256.000000000000000,252.000000000000000,14.000000000000000,11.000000000000000,321.000000000000000,119.000000000000000,24.000000000000000
50%,3.282269500000000e+06,0.000000000000000,7.306527500000000e+06,68.769000000000005,9678.000000000000000,361.000000000000000,150.000000000000000,226.000000000000000,299.000000000000000,87.000000000000000,...,166.000000000000000,15.000000000000000,341.000000000000000,472.000000000000000,252.000000000000000,14.000000000000000,11.000000000000000,321.000000000000000,149.000000000000000,24.000000000000000
75%,3.429904250000000e+06,0.000000000000000,1.124662000000000e+07,125.000000000000000,14184.000000000000000,512.000000000000000,150.000000000000000,226.000000000000000,330.000000000000000,87.000000000000000,...,225.000000000000000,15.000000000000000,427.000000000000000,533.000000000000000,486.500000000000000,14.000000000000000,15.000000000000000,371.000000000000000,169.000000000000000,32.000000000000000
max,3.577539000000000e+06,1.000000000000000,1.581113100000000e+07,31937.390999999999622,18396.000000000000000,600.000000000000000,231.000000000000000,237.000000000000000,540.000000000000000,102.000000000000000,...,229.000000000000000,29.000000000000000,671.000000000000000,661.000000000000000,854.000000000000000,44.000000000000000,26.000000000000000,548.000000000000000,216.000000000000000,32.000000000000000


In [11]:
train.shape

(590540, 434)

In [12]:
del train_identity, train_transaction

## Columns with missing values

In [13]:
train.isnull().any().sum()

414

## Categorical features

We see from the [data definition](https://www.kaggle.com/c/ieee-fraud-detection/data) that the following features are categorical:

### Transactions:

*    `ProductCD`
*    `card1` - `card6`
*    `addr1`, `addr2`
*    `P_emaildomain`
*    `R_emaildomain`
*    `M1` - `M9`

### Identity:


*    `DeviceType`
*    `DeviceInfo`
*    `id_12` - `id_38`

Let's verify this.

In [14]:
train.select_dtypes(include=['object']).columns

Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15',
       'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object')

### We need to follow up an a few of the purported columns

The following columns _were NOT_ object types:
    
* `card1`
* `card2`
* `card3`
* `card5`
* `id_13`
* `id_14`
* `id_16` - `id_22`
* `id_24` - `id_26`
* `id_32`
